In [1]:
import tensorflow as tf
import numpy as np
import math

Hint: Use dtype=tf.float64 if you want to have same precision as numpy for testing<br>
Hint: You migth wanna use tf.InteractiveSession for convenience

In [2]:
sess = tf.InteractiveSession()
fl = 'Fail'

1a: Create two random 0-d tensors x and y of any distribution. <br>
Create a TensorFlow object that returns x + y if x > y, and x - y otherwise. <br>
Hint: look up tf.cond() <br>
I do the first problem for you <br>

In [3]:
def task_1a_np(x, y):
    return np.where(x > y, x + y, x - y)

In [4]:
writer = tf.summary.FileWriter('./graphs/task1', sess.graph)
X = tf.placeholder(tf.float64, name='task1ax')
Y = tf.placeholder(tf.float64, name='task1ay')
addition = tf.add(X, Y)
subtraction = tf.subtract(X, Y)
condition = tf.greater(X, Y)
out = tf.cond(condition, lambda: addition, lambda: subtraction)

In [5]:
for xx, yy in np.random.uniform(size=(50, 2)):
    actual = out.eval(feed_dict={X:xx, Y:yy})
    expected = task_1a_np(xx, yy)
    if actual != expected:
        print(fl)
        break
else:
    print('Success')

Success


1b: Create two 0-d tensors x and y randomly selected from the range [-1, 1).<br>
Return x + y if x < y, x - y if x > y, 0 otherwise.<br>
Hint: Look up tf.case().<br>

In [6]:
def task_1b_np(x, y):
    return np.select(condlist=[x < y, x > y],
                     choicelist=[x + y, x - y],
                     default=0)

In [7]:
X = tf.placeholder(tf.float64)
Y = tf.placeholder(tf.float64)
out = tf.case([
    (tf.less(X, Y), lambda: X + Y),
    (tf.greater(X, Y), lambda: X - Y)
], lambda: tf.constant(0, dtype=tf.float64), exclusive=True)

In [8]:
for xx, yy in np.random.uniform(-1.0, 1.0, size=(50, 2)):
    actual = out.eval(feed_dict={X:xx, Y:yy})
    expected = task_1b_np(xx, yy)
    if actual != expected:
        print(fl)
        break
else:
    print('Success')

Success


1c: Create the tensor x of the value [[0, -2, -1], [0, 1, 2]] <br>
and y as a tensor of zeros with the same shape as x. <br>
Return a boolean tensor that yields Trues if x equals y element-wise. <br>
Hint: Look up tf.equal(). <br>

In [9]:
def task_1c_np():
    x = np.array([[0, -2, -1], [0, 1, 2]])
    y = np.zeros_like(x)
    return x == y

In [10]:
X = tf.constant( [[0, -2, -1], [0, 1, 2]])
Y = tf.zeros_like(X)
out = tf.equal(X, Y)

In [11]:
actual = out.eval()
expected = task_1c_np()
if not (actual == expected).all():
    print(fl)
else:
    print('Success')

Success


1d:<br>
Get the indices of elements in x whose values are greater than 30.<br>
Hint: Use tf.where().<br>
Then extract elements whose values are greater than 30.<br>
Hint: Use tf.gather().<br>

In [12]:
def task_1d_np(x):
    return x[x > 30].reshape(-1, 1)

In [13]:
X = tf.placeholder(tf.float64, shape=(None,))
out = tf.where(X > 30)
out = tf.gather(X, out)

In [14]:
for i in range(100):
    data = np.random.uniform(0, 60.0, 10000)
    actual = out.eval(feed_dict={X: data})
    expected = task_1d_np(data)
    if (actual != expected).any():
        print(fl)
        break
else:
    print('Success')

Success


1e: Create a diagnoal 2-d tensor of size 6 x 6 with the diagonal values of 1,<br>
2, ..., 6<br>
Hint: Use tf.range() and tf.diag().<br>

In [15]:
def task_1e_np():
    return np.diag(np.arange(1, 7))

In [16]:
out = tf.diag(tf.range(1, 7))
if (out.eval() != task_1e_np()).any():
    print(fl)
else:
    print('Success')

Success


1f: Create a random 2-d tensor of size 10 x 10 from any distribution.<br>
Calculate its determinant.<br>
Hint: Look at tf.matrix_determinant().<br>

In [17]:
def task_1f_np(x):
    return np.linalg.det(x)

In [18]:
for i in range(100):
    data = tf.truncated_normal((10, 10), dtype=tf.float64)
    out = tf.matrix_determinant(data)
    arr, actual = sess.run([data, out])
    expected = task_1f_np(arr)
    if abs(actual - expected) >= 1e-9: # Simply checking for `==` does not work
        print(fl)
        break
else:
    print('Success')

Success


1g: Create tensor x with value [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9].<br>
Return the unique elements in x<br>
Hint: use tf.unique(). Keep in mind that tf.unique() returns a tuple.<br>

In [19]:
def task_1g_np():
    x = [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9]
    _, idx = np.unique(x, return_index=True)
    return np.take(x, sorted(idx))

In [20]:
data = tf.constant([5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9], dtype=tf.int32)
out, _ = tf.unique(data)
if (out.eval() != task_1g_np()).any():
    print(fl)
else:
    print('Success')

Success


1h: Create two tensors x and y of shape 300 from any normal distribution,<br>
as long as they are from the same distribution.<br>
Use tf.cond() to return:<br>
- The mean squared error of (x - y) if the average of all elements in (x - y)<br>
  is negative, or<br>
- The sum of absolute value of all elements in the tensor (x - y) otherwise.<br>
Hint: see the Huber loss function in the lecture slides 3.<br>

In [21]:
def task_1h_np(x, y):
    average = np.mean(x - y)
    mse = np.mean((x - y) ** 2)
    asum = np.sum(np.abs(x - y))
    return mse if average < 0 else asum

In [22]:
x = tf.truncated_normal(shape=(300,), name='x', dtype=tf.float64)
y = tf.truncated_normal(shape=(300,), name='y', dtype=tf.float64)
condition = tf.less(tf.reduce_mean(x - y, name='mean_x-y'), 0, name='condition_less')
mse = tf.reduce_mean((x - y) ** 2)
abssum = tf.reduce_sum(tf.abs(x - y))
out = tf.cond(condition, lambda: mse, 
                         lambda: abssum)

In [23]:
for i in range(100):
    xx, yy, actual = sess.run([x, y, out])
    expected = task_1h_np(xx, yy)
    if abs(actual - expected) >= 1e-9:
        print(fl)
        print(actual)
        print(expected)
        break
else:
    print('Success')

Success


In [24]:
writer.close()